# Build a model for predicting house prices in Barcelona, using the Idealista dataset.

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
import random
import pickle

model_name = 'house_price_model.sav'

### Preprocess data by one-hot encoding categorical features

In [7]:
orig_data = pd.read_csv('barca_idealista.csv')
orig_data = orig_data.drop(labels=orig_data.columns[[0]], axis=1)
orig_data = orig_data.drop(labels='city', axis=1)

categorical_feature_mask = orig_data.dtypes == object
categorical_features = orig_data.columns[categorical_feature_mask].tolist()

one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
one_hot_encoder.fit(orig_data[categorical_features])

one_hot_categorical_data = pd.DataFrame(
    data=one_hot_encoder.transform(
        orig_data[categorical_features]
    ),
    columns=one_hot_encoder.get_feature_names_out(
        input_features=categorical_features
    )
)
data = pd.concat([orig_data, one_hot_categorical_data], axis=1)
data = data.drop(columns=categorical_features, axis=1)

labels = data.prices
data.drop(labels='prices', axis=1, inplace=True)

train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.25
)

In [4]:
all_categories = {}

for feature in categorical_features:
    orig_data[feature] = pd.Categorical(orig_data[feature])
    all_categories[feature] = list(orig_data[feature].cat.categories)

pickle.dump(all_categories, open('house_price_feature_categories.sav', 'wb'))

In [82]:
# Save one-hot encoder
pickle.dump(one_hot_encoder, open('house_price_encoder.sav', 'wb'))

### Fit different models

In [16]:
from sklearn.ensemble import RandomForestRegressor


model = RandomForestRegressor(random_state=0)
model.fit(train_data, train_labels)

RandomForestRegressor(random_state=0)

In [10]:
from sklearn.svm import SVR

model = SVR()
model.fit(train_data, train_labels)

SVR()

In [12]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(train_data, train_labels)

LinearRegression()

### Evaluate model

In [22]:
pred = model.predict(test_data)

print(f'mean abs error:          {mean_absolute_error(test_labels, pred)}')
print(f'r2 score:                {r2_score(test_labels, pred)}\n')

random_sample = random.randint(0, len(test_data))
print(f'Sample predicted price:  $ {int(model.predict(test_data.iloc[[random_sample]])[0])}')
print(f'Sample actual price:     $ {test_labels.iloc[random_sample]}')

mean abs error:          145946.2629276097
r2 score:                0.68690406372734

Sample predicted price:  $ 1325230
Sample actual price:     $ 1149000


In [23]:
# Save trained model
pickle.dump(model, open(model_name, 'wb'))

In [22]:
# Load trained model
model = pickle.load(open(model_name, 'rb'))